### This example shows simple single tool usage, if the LLM decides it needs tool it will call the tool else print regular response

In [1]:
import json
from openai import OpenAI

In [2]:
client = OpenAI(
    base_url='http://localhost:11434/v1',
    api_key='ollama',  # required, but unused
)

In [3]:
def calculate(operation, num1, num2):
    if operation == 'add':
        result = num1 + num2
    elif operation == 'subtract':
        result = num1 - num2
    elif operation == 'multiply':
        result = num1 * num2
    elif operation == 'divide':
        result = num1 / num2 if num2 != 0 else 'Error: Division by zero'
    else:
        result = 'Error: Unknown operation'
    
    return f"Tool calling response: The result of {operation}ing {num1} and {num2} is: {result}"

In [4]:
def handle_response(response):
    # Check for tool calls or direct message content
    if response.choices and response.choices[0].finish_reason == 'tool_calls':
        tool_calls = response.choices[0].message.tool_calls  # Accessing tool_calls directly
        for tool_call in tool_calls:
            if tool_call.function.name == 'calculate':
                # Parse the arguments from JSON string to dictionary
                arguments = json.loads(tool_call.function.arguments)
                result = calculate(arguments['operation'], arguments['num1'], arguments['num2'])
                print(result)
    elif response.choices:
        # Directly print the model's response if it's not a tool call
        print("Model response:", response.choices[0].message.content)

In [5]:
# Provide a calculator tool to the model
tools = [{
    'type': 'function',
    'function': {
        'name': 'calculate',
        'description': 'Perform a basic arithmetic operation',
        'parameters': {
            'type': 'object',
            'properties': {
                'operation': {
                    'type': 'string',
                    'description': 'The operation to perform (add, subtract, multiply, divide)',
                },
                'num1': {
                    'type': 'number',
                    'description': 'First number',
                },
                'num2': {
                    'type': 'number',
                    'description': 'Second number',
                },
            },
            'required': ['operation', 'num1', 'num2'],
        },
    },
}]

In [6]:
query1 = "What is the capital of France?"

In [9]:
response = client.chat.completions.create(
    model='qwen2.5:14b-instruct-q5_K_M',
    messages=[{'role': 'user', 'content': query1}],
    tools=tools,
    tool_choice="auto"
)

# Handle the response, which includes the function calling logic
handle_response(response)

Model response: The capital of France is Paris.


In [10]:
query2 = "What is the product of 78 and 54?"

In [11]:
response = client.chat.completions.create(
    model='qwen2.5:14b-instruct-q5_K_M',
    messages=[{'role': 'user', 'content': query2}],
    tools=tools,
    tool_choice="auto"
)

# Handle the response, which includes the function calling logic
handle_response(response)

Tool calling response: The result of multiplying 78 and 54 is: 4212
